# Day 6 - Lab 1: Building RAG Systems

**Objective:** Build a RAG (Retrieval-Augmented Generation) system orchestrated by LangGraph, scaling in complexity from a simple retriever to a multi-agent team that includes a grader and a router.

**Estimated Time:** 180 minutes

**Introduction:**
Welcome to Day 6! Today, we build one of the most powerful and common patterns for enterprise AI: a system that can answer questions about your private documents. We will use LangGraph to create a 'research team' of AI agents. Each agent will have a specific job, and LangGraph will act as the manager, orchestrating their collaboration to find the best possible answer.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We need several libraries for this lab. `langgraph` is the core orchestrator, `langchain` provides the building blocks, `faiss-cpu` is for our vector store, and `pypdf` is for loading documents.

**Model Selection:**
For RAG and agentic workflows, models with strong instruction-following and reasoning are best. `gpt-4.1`, `o3`, or `gemini-2.5-pro` are excellent choices.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `load_artifact()`: To read the project documents that will form our knowledge base.

In [1]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

import importlib
def install_if_missing(package):
    try:
        importlib.import_module(package)
    except ImportError:
        print(f"{package} not found, installing...")
        import subprocess
        subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", package])

install_if_missing('langgraph')
install_if_missing('langchain')
install_if_missing('langchain_community')
install_if_missing('langchain_openai')
install_if_missing('faiss-cpu')
install_if_missing('pypdf')

from utils import setup_llm_client, load_artifact
client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

faiss-cpu not found, installing...
✅ LLM Client configured: Using 'openai' with model 'gpt-4.1'


## Step 2: Building the Knowledge Base

An agent is only as smart as the information it can access. We will create a vector store containing all the project artifacts we've created so far. This will be our agent's 'knowledge base'.

In [2]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.documents import Document

def create_knowledge_base(file_paths):
    """Loads documents from given paths and creates a FAISS vector store.""" 
    all_docs = []
    for path in file_paths:
        full_path = os.path.join(project_root, path)
        if os.path.exists(full_path):
            loader = TextLoader(full_path)
            docs = loader.load()
            for doc in docs:
                doc.metadata={"source": path} # Add source metadata
            all_docs.extend(docs)
        else:
            print(f"Warning: Artifact not found at {full_path}")

    if not all_docs:
        print("No documents found to create knowledge base.")
        return None

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    splits = text_splitter.split_documents(all_docs)
    
    print(f"Creating vector store from {len(splits)} document splits...")
    vectorstore = FAISS.from_documents(documents=splits, embedding=OpenAIEmbeddings())
    return vectorstore.as_retriever()

all_artifact_paths = ["artifacts/day1_prd.md", "artifacts/schema.sql", "artifacts/adr_001_database_choice.md"]
retriever = create_knowledge_base(all_artifact_paths)


Creating vector store from 13 document splits...


## Step 3: The Challenges

### Challenge 1 (Foundational): A Simple RAG Graph

**Task:** Build a simple LangGraph with two nodes: one to retrieve documents and one to generate an answer.

> **Tip:** Think of `AgentState` as the shared 'whiteboard' for your agent team. Every agent (or 'node' in the graph) can read from and write to this state, allowing them to pass information to each other as they work on a problem.

**Instructions:**
1.  Define the state for your graph using a `TypedDict`. It should contain keys for `question` and `documents`.
2.  Create a "Retriever" node. This is a Python function that takes the state, uses the `retriever` to get relevant documents, and updates the state with the results.
3.  Create a "Generator" node. This function takes the state, creates a prompt with the question and retrieved documents, calls the LLM, and stores the answer.
4.  Build the `StateGraph`, add the nodes, and define the edges (`RETRIEVE` -> `GENERATE`).
5.  Compile the graph and invoke it with a question about your project.

**Expected Quality:** A functional graph that can answer a simple question (e.g., "What is the purpose of this project?") by retrieving context from the project artifacts.

In [19]:
# TODO: Write the code for the single-agent RAG system using LangGraph.
# This will involve defining the state, the nodes, and the graph itself.
from typing import TypedDict, List
from langgraph.graph import StateGraph, END
import textwrap
class State(TypedDict):
    """State for the RAG system."""
    question: str
    documents: List[Document]
    answer: str

def retriever_node(state):
    """
    Retriever node for LangGraph RAG system.
    Takes the state dict, uses the retriever to get relevant documents,
    and updates the state with the retrieved documents.
    """
    print("Retrieving documents...")
    question = state["question"]
    # Use the retriever to get relevant documents
    docs = retriever.get_relevant_documents(question)
    # Update the state with the retrieved documents
    return {"documents": docs}

def generator_node(state):
    """ Generator node for LangGraph RAG system.
    Takes the state dict, uses the LLM to generate an answer based on the question and documents,
    and updates the state with the generated answer.
    """
    print("Generating answer...")
    question = state["question"]
    documents =state["documents"] 
    prompt = f"""You are an AI assistant that answers questions based on the provided documents.
    Question: {question}
    Documents: {documents}
    """
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
    )
    answer = response.choices[0].message.content.strip()

    if answer is not None:
        wrapped = textwrap.fill(str(answer), width=200)
        print(f"Generated answer: {wrapped}")
    else:
        print("No answer generated.")

    print ("Documents used for answer generation:")
    for idx, doc in enumerate(documents, start=1):
        wrapped_doc = textwrap.fill(str(doc.page_content), width=200)
        print(f"Document {idx}: {wrapped_doc}")
    # Typed dictionary stuff
    return {"answer": answer}

# Define the state graph for the RAG system
print("Creating the state graph...")
graph = StateGraph(State)
print("adding nodes to the graph...")
graph.add_node("RETRIEVE", retriever_node)
graph.add_node("GENERATE", generator_node)

print("adding edges to the graph...")
graph.set_entry_point("RETRIEVE")
graph.add_edge("RETRIEVE", "GENERATE")

print("compiling the graph...")
app = graph.compile()

app.invoke(State(question="What is the purpose of this project"))




Creating the state graph...
adding nodes to the graph...
adding edges to the graph...
compiling the graph...
Retrieving documents...
Generating answer...
Generated answer: The purpose of this project is to develop a new hire onboarding tool that includes an AI-powered semantic search feature to help personalize learning paths for new employees. The tool aims to make the
onboarding process more effective by leveraging semantic search capabilities, allowing new hires to easily find relevant information and resources. The project specifically chooses to use PostgreSQL
with the pgvector extension for semantic search, prioritizing reduced learning curve, operational simplicity, and cost-effectiveness by building on familiar and existing database infrastructure. The
ultimate goal is to create an onboarding experience that is efficient, scalable, and tailored to individual needs through AI-driven personalization.
Documents used for answer generation:
Document 1: - **Future Work:**   - Monitor

{'question': 'What is the purpose of this project',
 'documents': [Document(id='b31691e6-9045-4580-9bf9-ec4dab2691ae', metadata={'source': 'artifacts/adr_001_database_choice.md'}, page_content="- **Future Work:**\n  - Monitor the system's performance and scalability as the dataset grows to determine if a transition to a specialized vector database becomes necessary.\n  - Evaluate the need for potential architectural adjustments to accommodate high-dimensional vector data and improve search performance if required.\n```"),
  Document(id='2c0616e0-5177-4ebb-bd96-a91e90ccc75e', metadata={'source': 'artifacts/day1_prd.md'}, page_content='FutureWork(description="AI-powered personalized learning paths for new hires.")\n    ],\n    appendix=[\n        AppendixItem(type="Open Question", description="Which team will be responsible for maintaining the content in the document repository?"),\n        AppendixItem(type="Dependency", description="The final UI design mockups are required from the Des

### Challenge 2 (Intermediate): A Graph with a Grader Agent

**Task:** Add a second agent to your graph that acts as a "Grader," deciding if the retrieved documents are relevant enough to answer the question.

> **What is a conditional edge?** It's a decision point. After a node completes its task (like our 'Grader'), the conditional edge runs a function to decide which node to go to next. This allows your agent to change its plan based on new information.

**Instructions:**
1.  Keep your `RETRIEVE` and `GENERATE` nodes from the previous challenge.
2.  Create a new "Grader" node. This function takes the state (question and documents) and calls an LLM with a specific prompt: "Based on the question and the following documents, is the information sufficient to answer the question? Answer with only 'yes' or 'no'."
3.  Add a **conditional edge** to your graph. After the `RETRIEVE` node, the graph should go to the `GRADE` node. After the `GRADE` node, it should check the grader's response. If 'yes', it proceeds to the `GENERATE` node. If 'no', it goes to an `END` node, concluding that it cannot answer the question.

**Expected Quality:** A more robust graph that can gracefully handle cases where its knowledge base doesn't contain the answer, preventing it from hallucinating.

In [18]:
# TODO: Write the code for the two-agent system with a Grader and conditional edges.
def retriever_node(state):
    """
    Retriever node for LangGraph RAG system.
    Takes the state dict, uses the retriever to get relevant documents,
    and updates the state with the retrieved documents.
    """
    print("Retrieving documents...")
    question = state["question"]
    # Use the retriever to get relevant documents
    docs = retriever.get_relevant_documents(question)
    # Update the state with the retrieved documents
    return {"documents": docs}

def generator_node(state):
    """ Generator node for LangGraph RAG system.
    Takes the state dict, uses the LLM to generate an answer based on the question and documents,
    and updates the state with the generated answer.
    """
    print("Generating answer...")
    question = state["question"]
    documents =state["documents"] 
    prompt = f"""You are an AI assistant that answers questions based on the provided documents.
    Question: {question}
    Documents: {documents}
    """
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
    )
    answer = response.choices[0].message.content.strip()
    # Typed dictionary stuff
    return {"answer": answer}

def grader_node(state):
    """ Generator node for LangGraph RAG system.
    Takes the state dict, uses the LLM to generate an answer based on the question and documents,
    and updates the state with the generated answer.
    """
    print("Grading the answer...")
    question = state["question"]
    documents =state["documents"] 
    prompt = f"""Based on the question and the following documents, is the information sufficient to answer the question? Answer with only 'yes' or 'no'.
    Question: {question}
    Documents: {documents}
    """
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role":"system", "content": "You are a grading AI that determines if the provided documents are sufficient to answer the question."},
            {"role": "user", "content": prompt}],
    )
    graded_answer = response.choices[0].message.content.strip()
    if graded_answer.lower() == "yes":
        return {"documents": documents}
    else:
        return {"documents": []}
    
def rewrite_question(state):
    """Rewrite question node for LangGraph RAG system.
    This node is used to rewrite the question if the Grader node returns 'no'.
    """
    print("Rewriting question...")
    question = state["question"]
    prompt = f"""Rewrite the following question to make it more specific and clear:
    Question: {question}
    """
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
    )
    rewritten_question = response.choices[0].message.content.strip()
    return {"question": rewritten_question}

def continue_node(state):
    """Continue node for LangGraph RAG system.
    This node is used to continue the process if the Grader node returns 'yes'.
    """
    print("Continuing to generate answer...")
    if state["documents"]:
        print("Documents are sufficient, proceeding to answer generation.")
        return "generate"
    else:
        print("Documents are not sufficient, ending process.")
        return "rewrite"

# Define the state graph for the RAG system
print("Creating the state graph...")
graph = StateGraph(State)
print("adding nodes to the graph...")
graph.add_node("RETRIEVE", retriever_node)
graph.add_node("GENERATE", generator_node)
graph.add_node("GRADER", grader_node)
graph.add_node("REWRITE", rewrite_question)


print("adding edges to the graph...")
graph.set_entry_point("RETRIEVE")
graph.add_edge("RETRIEVE", "GRADER")
graph.add_conditional_edges(
    "GRADER",
    continue_node,
    {
        "rewrite": "REWRITE",
        "generate": "GENERATE"
    }
)
graph.add_edge("REWRITE", "RETRIEVE")
graph.add_edge("GENERATE", END)

app = graph.compile()
app.invoke(State(question="What project do?"))





Creating the state graph...
adding nodes to the graph...
adding edges to the graph...
Retrieving documents...
Grading the answer...
Continuing to generate answer...
Documents are sufficient, proceeding to answer generation.
Generating answer...


{'question': 'What project doing?',
 'documents': [Document(id='d107dfb5-c58f-4f5a-a596-35db881d8cad', metadata={'source': 'artifacts/adr_001_database_choice.md'}, page_content='## Context\n- The project involves developing a new hire onboarding tool that requires a semantic search feature. The decision to use PostgreSQL with the `pgvector` extension over specialized vector databases such as ChromaDB, FAISS, or Weaviate involves several considerations.\n- PostgreSQL is a well-established relational database system, familiar to many development teams, which can reduce the initial learning curve and integration time.\n- Leveraging PostgreSQL with `pgvector` allows the use of existing database infrastructure, minimizing operational complexity by avoiding the need to manage a separate system for vector storage and search.\n- Cost-effectiveness is another important factor, as integrating `pgvector` into a current PostgreSQL setup may incur fewer additional costs than adopting a new technolo

### Challenge 3 (Advanced): A Multi-Agent Research Team

**Task:** Build a sophisticated "research team" of specialized agents that includes a router to delegate tasks to the correct specialist.

**Instructions:**
1.  **Specialize your retriever:** Create two separate retrievers. One for the PRD (`prd_retriever`) and one for the technical documents (`tech_retriever` for schema and ADRs).
2.  **Define the Agents:**
    * `ProjectManagerAgent`: This will be the entry point and will act as a router. It uses an LLM to decide whether the user's question is about product requirements or technical details, and routes to the appropriate researcher.
    * `PRDResearcherAgent`: A node that uses the `prd_retriever`.
    * `TechResearcherAgent`: A node that uses the `tech_retriever`.
    * `SynthesizerAgent`: A node that takes the collected documents from either researcher and synthesizes a final answer.
3.  **Build the Graph:** Use conditional edges to orchestrate the flow: The entry point is the `ProjectManager`, which then routes to either the `PRD_RESEARCHER` or `TECH_RESEARCHER`. Both of those nodes should then route to the `SYNTHESIZE` node, which then goes to the `END`.

**Expected Quality:** A highly advanced agentic system that mimics a real-world research workflow, including a router and specialist roles, to improve the accuracy and efficiency of the RAG process.

In [21]:
# 1. Imports and State
class State(TypedDict):
    """State for the RAG system."""
    question: str
    documents: List[Document]
    answer: str
    route: str 

prd_artifact_paths = ["artifacts/day1_prd.md"]
tech_artifact_paths = ["artifacts/schema.sql", "artifacts/adr_001_database_choice.md"]

prd_retriever = create_knowledge_base(prd_artifact_paths)
tech_retriever = create_knowledge_base(tech_artifact_paths)

# 2. Define agent nodes
def project_manager_node(state):
    """
    Router node that decides which researcher to route to based on the question.
    Uses simple keyword logic; can be replaced with an LLM for more advanced routing.
    """
    question = state["question"]
    if "requirement" in question.lower() or "prd" in question.lower():
        print("Routing to PRDResearcherAgent...")
        return {"route": "PRD_RESEARCHER"}
    else:
        print("Routing to TechResearcherAgent...")
        return {"route": "TECH_RESEARCHER"}


def prd_researcher_node(state):
    """
    Uses prd_retriever to fetch relevant PRD documents for the question.
    """
    print("Retrieving PRD documents...")
    question = state["question"]
    prd_docs = prd_retriever.get_relevant_documents(question)
    return {"documents": prd_docs}


def tech_researcher_node(state):
    """
    Uses tech_retriever to fetch relevant technical documents for the question.
    """
    print("Retrieving technical documents...")
    question = state["question"]
    tech_docs = tech_retriever.get_relevant_documents(question)
    return {"documents": tech_docs}


def synthesizer_node(state):
    """
    Synthesizes a final answer from the documents retrieved by either researcher.
    Calls the LLM with a prompt containing the question and the retrieved documents.
    """
    print("Synthesizing final answer...")
    question = state["question"]
    documents = state["documents"]
    prompt = f"""You are an AI assistant that synthesizes answers from the following documents.\nQuestion: {question}\nDocuments: {documents}"""
    response = client.chat.completions.create(
        model=model_name,
        messages=[{"role": "user", "content": prompt}],
    )
    answer = response.choices[0].message.content.strip()
    print(f"Final synthesized answer: {answer}")
    return {"answer": answer}

# 3. Build the graph
print("Creating the multi-agent RAG state graph...")
graph = StateGraph(State)
graph.add_node("PROJECT_MANAGER", project_manager_node)
graph.add_node("PRD_RESEARCHER", prd_researcher_node)
graph.add_node("TECH_RESEARCHER", tech_researcher_node)
graph.add_node("SYNTHESIZE", synthesizer_node)

graph.set_entry_point("PROJECT_MANAGER")
graph.add_conditional_edges(
    "PROJECT_MANAGER",
    lambda state: state["route"],
    {
        "PRD_RESEARCHER": "PRD_RESEARCHER",
        "TECH_RESEARCHER": "TECH_RESEARCHER"
    }
)
graph.add_edge("PRD_RESEARCHER", "SYNTHESIZE")
graph.add_edge("TECH_RESEARCHER", "SYNTHESIZE")
graph.add_edge("SYNTHESIZE", END)

app = graph.compile()

# Example invocation
result = app.invoke(State(question="What are the product requirements?"))
print(result)


Creating vector store from 7 document splits...
Creating vector store from 6 document splits...
Creating the multi-agent RAG state graph...
Routing to PRDResearcherAgent...
Retrieving PRD documents...
Synthesizing final answer...
Final synthesized answer: The product requirements, synthesized from the provided documents, are as follows:

**1. Executive Summary and Vision**
- The product aims to streamline onboarding for new hires by providing a centralized, user-friendly platform.
- Vision: Enhance productivity and engagement for new hires by reducing onboarding friction.

**2. Problem Statement**
- New hires currently face a fragmented and overwhelming onboarding experience.

**3. User Personas and Scenarios**
- The New Hire: Faces confusion with multiple onboarding platforms.
- The Hiring Manager: Spends excessive time answering repetitive questions.
- The HR Coordinator: Handles high volumes of support tickets.

**4. Goals and Metrics**
- (Specific metrics/goals are referenced, but 

## Lab Conclusion

Incredible work! You have now built a truly sophisticated AI system. You've learned how to create a knowledge base for an agent and how to use LangGraph to orchestrate a team of specialized agents to solve a complex problem. You progressed from a simple RAG chain to a system that includes quality checks (the Grader) and intelligent task delegation (the Router). These are the core patterns for building production-ready RAG applications.

> **Key Takeaway:** LangGraph allows you to define complex, stateful, multi-agent workflows as a graph. Using nodes for agents and conditional edges for decision-making enables the creation of sophisticated systems that can reason, delegate, and collaborate to solve problems more effectively than a single agent could alone.